## Writing a Simple Action Client
This tutorial covers using the `action_client` library to create a Fibonacci simple action client in Python.

In [ ]:
import rospy
import time
from __future__ import print_function

In [ ]:
from actionlib import SimpleActionClient
from actionlib_tutorials.msg import FibonacciAction, FibonacciGoal

The action specification generates several messages for sending goals, receiving feedback, etc... This line imports the generated messages.

In [ ]:
rospy.init_node('fibonacci_client')

We create the client node.

In [ ]:
client = SimpleActionClient('fibonacci', FibonacciAction)

The action client and server communicate over a set of topics, described in [the actionlib protocol](http://wiki.ros.org/actionlib/DetailedDescription). The action name describes the namespace containing these topics, and the action specification message describes what messages should be passed along these topics.

In [ ]:
client.wait_for_server()

Sending goals before the action server comes up would be useless. This line waits until we are connected to the action server.

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal)

Creates a goal and sends it to the action server.

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

The action server will process the goal and eventually terminate. We want the result from the termination, but we wait until the server has finished with the goal.

## Processing feedback

We define a callback that will be executed when a feedback message is received. The callback just print the contents of the feedback:

In [ ]:
def feedback_cb(feedback):
    print("[Feedback] Sequence:", feedback.sequence)

Now let's send a goal.

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal, feedback_cb=feedback_cb)

While waiting, we receive one feedback update per second.

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

We can now receive the successful result.

## Cancelling a request

Now let's try preempting a goal. These lines will send a goal, cause the client to sleep briefly, then request that the server preempt the goal:

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal)
time.sleep(1.5)
client.cancel_goal()

The client can still get the partial result of the task.

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

## Preempting a previous goal

The `SimpleActionServer` implements a single goal policy: accepting a new goal implies successful preemption of any old goal.

In [ ]:
goal = FibonacciGoal(order=5)
client.send_goal(goal)
time.sleep(1.5)
new_goal = FibonacciGoal(order=8)
client.send_goal(new_goal)

We will get the result of the last goal sent.

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

## Action aborted by server

Now let's trigger a server-side abort: the order is higher than the server threshold, and the goal will be aborted.

In [ ]:
goal = FibonacciGoal(order=105)
client.send_goal(goal)

When asking for the result, we get a status message from the server.

In [ ]:
client.wait_for_result()
result = client.get_result()
print('[Result]    State: %d' % client.get_state())
print('[Result]   Status: %s' % client.get_goal_status_text())
print('[Result] Sequence:', result.sequence)

This tutorial has covered `actions`, a powerful communications tool that is commonly used in ROS systems. For additional details, consult the [`actionlib` API documentation](http://wiki.ros.org/actionlib).

For going back to the main page, please close the other tabs and click on the following link:
[Go back to the main page](../../README.ipynb)